In [1]:
import onnx
import numpy as np
from qonnx.util.basic import qonnx_make_model
from finn.util.visualization import showInNetron,showSrc
import onnxruntime as rt
from qonnx.util.basic import qonnx_make_model
from onnx.helper import make_tensor_value_info, make_node, make_graph, make_model, make_tensor
from onnx import numpy_helper
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
import finn.core.onnx_exec as oxe
import torch
from brevitas.nn import QuantReLU, QuantIdentity
from finn.transformation.streamline import Streamline
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline import RoundAndClipThresholds
from qonnx.core.datatype import DataType
from qonnx.transformation.qcdq_to_qonnx import QCDQToQuant
import finn.core.onnx_exec as oxe

In [ ]:
#This is the model exported from brevitas

# change the finn path to your local path


showInNetron('./acm-transactions-cps-model.onnx')

FileNotFoundError: [Errno 2] No such file or directory: './acm-transactions-cps-model.onnx'

In [3]:
# id_path = "full_ids_onnx_export_ready_for_streamlining_2.onnx"#This is the best file till now. Testing the cell quantized one below.
id_path = "./onnx_definition_acm_trans_cps.onnx"
model_qcdq = ModelWrapper(id_path)
model = model_qcdq.transform(QCDQToQuant())
model = model.transform(InferShapes())
model.save("./quant_threshold_transform.onnx") 
showInNetron('./onnx_definition_acm_trans_cps.onnx')

Stopping http://0.0.0.0:8081
Serving './onnx_definition_acm_trans_cps.onnx' at http://0.0.0.0:8081


In [4]:
showInNetron('./quant_threshold_transform.onnx')

Stopping http://0.0.0.0:8081
Serving './quant_threshold_transform.onnx' at http://0.0.0.0:8081


In [5]:
in_X = np.ones([10,1],dtype=np.float32).reshape([10,1])
in_X[0][0] = 0
in_X[1][0] = 1
in_X[2][0] = 2
in_X[3][0] = 3
in_X[4][0] = 4
in_X[5][0] = 5
in_X[6][0] = 6
in_X[7][0] = 7
in_X[8][0] = 8
in_X[9][0] = 9
# in_h_t_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
# in_c_t_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
in_h_t_1 = np.ones([20,1],dtype=np.float32).reshape([20,1])
in_c_t_1 = np.ones([20,1],dtype=np.float32).reshape([20,1])
in_h_t_1[0][0] = 15
in_c_t_1[0][0] = 12
# in_h_t_1 = np.array([0,0.8614,0,0,0,0.8614,-0.8614,-0.8614,0,0,0.8332,0,-0.8614,0.8332,0.8614,0.8614,0,0.8614,0,0.8614],dtype=np.float32).reshape([20,1])
# in_c_t_1 = np.array([0,1.2486,1.2486,-1.2486,0,1.2486,-1.2486,-1.2486,0,0,1.2486,0,-1.2486,-1.2486,1.2486,1.2486,1.2486,1.2486,1.2486,1.2486],dtype=np.float32).reshape([20,1])
input_dict = {}
input_dict["X"] = in_X
input_dict["h_t-1"] = in_h_t_1
input_dict["c_t-1"] = in_c_t_1

output_dict_qonnx = oxe.execute_onnx(model, input_dict,return_full_exec_context=True)
print(output_dict_qonnx)
QONNX_out = np.array(output_dict_qonnx.get("dql_hidden_out"))
print(QONNX_out)
QONNX_out_ct = np.array(output_dict_qonnx.get("c_t_out"))
print(QONNX_out_ct)

{'h_t-1': array([[15.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32), 'c_t-1': array([[12.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32), 'X': array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.],
       [8.],
       [9.]], dtype=float32), 'out_matmul_fc3': array([[  7.192231 ],
       [-11.698284 ],
       [ -3.6276917],
       [ -9.907965 ],
       [-12.317261 ]], dtype=float32), 'dql_hidden_out': array([[-0.42216498],
       [-0.6332475 ],
       [ 0.03518042],
      

In [ ]:
# check_1 = np.array(output_dict_qonnx.get("h_t_inter"))#sixth_dql1
# print(check_1)
# check_3 = np.array(output_dict_qonnx.get("dql_hidden_out"))#sixth_dql1
# print(check_3)
# print(check_3 - check_1)

In [6]:
finn_model = model.transform(ConvertQONNXtoFINN())
finn_model.save("./quant_threshold_finn_onnx.onnx") 
showInNetron('./quant_threshold_finn_onnx.onnx')

Done function-1
Done function-2
Done function-3


/home/temporary/Desktop/qlstm_finn/finn/deps/qonnx/src/qonnx/transformation/gemm_to_matmul.py:57: UserWarning: The GemmToMatMul transformation only offers explicit support for version 9 of the Gemm node, but the ONNX version of the supplied model is 11. Thus the transformation may fail or return incomplete results.
  warnings.warn(


Done function-4
Done function-5
In adder bias function
In adder bias function
In adder bias function
In adder bias function
In adder bias function
In adder bias function
In adder bias function
Sigmoid
Sigmoid1 0.015187747
In adder bias function
Tanh
Tanh1 0.03136686
In adder bias function
Sigmoid
Sigmoid3 0.014742396
In adder bias function
Sigmoid
Sigmoid2 0.015358801
In adder bias function
In adder bias function
In adder bias function
In adder bias function
In adder bias function
Tanh
Tanh2 0.023423504
In adder bias function
In adder bias function
In adder bias function
Relu
In adder bias function
Relu
In adder bias function
Relu
In adder bias function
Done function-6
Done function-7
Done function-8
Done function-9
Stopping http://0.0.0.0:8081
Serving './quant_threshold_finn_onnx.onnx' at http://0.0.0.0:8081


In [7]:
in_X = np.ones([10,1],dtype=np.float32).reshape([10,1])
in_X[0][0] = 0
in_X[1][0] = 1
in_X[2][0] = 2
in_X[3][0] = 3
in_X[4][0] = 4
in_X[5][0] = 5
in_X[6][0] = 6
in_X[7][0] = 7
in_X[8][0] = 8
in_X[9][0] = 9
# in_h_t_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
# in_c_t_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
in_h_t_1 = np.ones([20,1],dtype=np.float32).reshape([20,1])
in_c_t_1 = np.ones([20,1],dtype=np.float32).reshape([20,1])
in_h_t_1[0][0] = 15
in_c_t_1[0][0] = 12
# in_h_t_1 = np.array([0,0.8614,0,0,0,0.8614,-0.8614,-0.8614,0,0,0.8332,0,-0.8614,0.8332,0.8614,0.8614,0,0.8614,0,0.8614],dtype=np.float32).reshape([20,1])
# in_c_t_1 = np.array([0,1.2486,1.2486,-1.2486,0,1.2486,-1.2486,-1.2486,0,0,1.2486,0,-1.2486,-1.2486,1.2486,1.2486,1.2486,1.2486,1.2486,1.2486],dtype=np.float32).reshape([20,1])
input_dict = {}
input_dict["X"] = in_X
input_dict["h_t-1"] = in_h_t_1
input_dict["c_t-1"] = in_c_t_1

output_dict_finn = oxe.execute_onnx(finn_model, input_dict,return_full_exec_context=True)#return_full_exec_context=True
print(output_dict_finn)
FINN_ONNX_out = np.array(output_dict_finn.get("h_t_inter"))
print(FINN_ONNX_out)

/home/temporary/Desktop/qlstm_finn/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor 7F5Kn5 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/temporary/Desktop/qlstm_finn/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor MD2HQB can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/temporary/Desktop/qlstm_finn/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor 8m2p5X can't be represented with the set datatype annotation (INT6), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/temporary/Desktop/qlstm_finn/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor XoDSRT can't be represented with the set datatype annotation (INT6), they will be rounded to match the datatype annotatio

{'h_t-1': array([[15.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32), 'c_t-1': array([[12.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32), 'X': array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.],
       [8.],
       [9.]], dtype=float32), 'out_matmul_fc3': array([[  7.1922307],
       [-11.698284 ],
       [ -3.6276915],
       [ -9.907965 ],
       [-12.317261 ]], dtype=float32), 'dql_hidden_out': array([[-0.42216498],
       [-0.6332475 ],
       [ 0.03518042],
      

In [8]:
check_2 = np.array(output_dict_finn.get("c_t_out"))#sixth_dql1
print(check_2)
check_1 = np.array(output_dict_qonnx.get("c_t_out"))#sixth_dql1
print(check_1)
print("----------------------------------------------")
# check_3 = np.array(output_dict_qonnx.get("dql_hidden_out"))#sixth_dql1
# print(check_3)
print(check_2 - check_1)

[[-0.47843987]
 [-0.7774648 ]
 [ 0.9269772 ]
 [ 0.9269772 ]
 [ 0.9269772 ]
 [ 0.17941496]
 [ 0.02990249]
 [-0.17941496]
 [ 0.9269772 ]
 [ 0.5980498 ]
 [ 0.3887324 ]
 [-0.86717224]
 [-0.56814736]
 [ 0.5980498 ]
 [ 0.9269772 ]
 [ 0.9269772 ]
 [-0.20931745]
 [ 0.47843987]
 [ 0.9269772 ]
 [ 0.9269772 ]]
[[-0.47843987]
 [-0.7774648 ]
 [ 0.9269772 ]
 [ 0.9269772 ]
 [ 0.9269772 ]
 [ 0.17941496]
 [ 0.02990249]
 [-0.17941496]
 [ 0.9269772 ]
 [ 0.62795234]
 [ 0.3887324 ]
 [-0.86717224]
 [-0.56814736]
 [ 0.5980498 ]
 [ 0.9269772 ]
 [ 0.9269772 ]
 [-0.20931745]
 [ 0.47843987]
 [ 0.9269772 ]
 [ 0.9269772 ]]
----------------------------------------------
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.02990252]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


In [9]:
x = np.array(output_dict_qonnx.get("dql_hidden_out")) #i_t_dql1
print(x)
x1 = np.array(output_dict_finn.get("dql_hidden_out")) #i_t_dql1
print(x1)
print(x-x1)

[[-0.42216498]
 [-0.6332475 ]
 [ 0.03518042]
 [ 0.70360833]
 [ 0.70360833]
 [ 0.10554124]
 [ 0.03518042]
 [-0.17590208]
 [ 0.10554124]
 [ 0.52770627]
 [ 0.        ]
 [-0.6684279 ]
 [-0.49252582]
 [ 0.52770627]
 [ 0.03518042]
 [ 0.6332475 ]
 [-0.21108249]
 [ 0.42216498]
 [ 0.70360833]
 [ 0.70360833]]
[[-0.42216498]
 [-0.6332475 ]
 [ 0.03518042]
 [ 0.70360833]
 [ 0.70360833]
 [ 0.10554124]
 [ 0.03518042]
 [-0.17590208]
 [ 0.10554124]
 [ 0.52770627]
 [ 0.        ]
 [-0.6684279 ]
 [-0.49252582]
 [ 0.52770627]
 [ 0.03518042]
 [ 0.6332475 ]
 [-0.21108249]
 [ 0.42216498]
 [ 0.70360833]
 [ 0.70360833]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [ ]:
scale = 0.03155536949634552
difference = x - x1
difference = difference/scale
print(difference)

In [ ]:
#MT = np.array(output_dict.get("FqSx1q"))
M1 = np.array(output_dict_qonnx.get("c_t_dql2"))
#print(M1)
M1F = np.array(output_dict_finn.get("c_t_dql2"))
#print(M1F)
diff = M1F - M1
print(diff)

In [ ]:
#Comparing FINN-ONNX and QONNX output
#print(FINN_ONNX_out.shape)
scale = 0.03176329657435417
difference = FINN_ONNX_out - QONNX_out
difference = difference/scale
print(difference)

In [10]:
from finn.transformation.streamline import Streamline
showSrc(Streamline)

class Streamline(Transformation):
    """Apply the streamlining transform, see arXiv:1709.04060."""

    def apply(self, model):
        streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),
            MoveMulPastMaxPool(),
            AbsorbAddIntoMultiThreshold(),
            FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
        for tr

In [11]:
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants
#streamlined_model = finn_model.transform(Streamline())
finn_model = finn_model.transform(InferShapes())
finn_model = finn_model.transform(FoldConstants())
finn_model = finn_model.transform(GiveUniqueNodeNames())
finn_model = finn_model.transform(GiveReadableTensorNames())
finn_model = finn_model.transform(InferDataTypes())
finn_model = finn_model.transform(RemoveStaticGraphInputs())
finn_model.save('./tidy_up_model.onnx')
showInNetron('./tidy_up_model.onnx')#,,localhost_url='xirxlabs53'

Stopping http://0.0.0.0:8081
Serving './tidy_up_model.onnx' at http://0.0.0.0:8081


In [12]:
in_X = np.ones([10,1],dtype=np.float32).reshape([10,1])
in_X[0][0] = 0
in_X[1][0] = 1
in_X[2][0] = 2
in_X[3][0] = 3
in_X[4][0] = 4
in_X[5][0] = 5
in_X[6][0] = 6
in_X[7][0] = 7
in_X[8][0] = 8
in_X[9][0] = 9
# in_h_t_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
# in_c_t_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
in_h_t_1 = np.ones([20,1],dtype=np.float32).reshape([20,1])
in_c_t_1 = np.ones([20,1],dtype=np.float32).reshape([20,1])
in_h_t_1[0][0] = 15
in_c_t_1[0][0] = 12
# in_h_t_1 = np.array([0,0.8614,0,0,0,0.8614,-0.8614,-0.8614,0,0,0.8332,0,-0.8614,0.8332,0.8614,0.8614,0,0.8614,0,0.8614],dtype=np.float32).reshape([20,1])
# in_c_t_1 = np.array([0,1.2486,1.2486,-1.2486,0,1.2486,-1.2486,-1.2486,0,0,1.2486,0,-1.2486,-1.2486,1.2486,1.2486,1.2486,1.2486,1.2486,1.2486],dtype=np.float32).reshape([20,1])
input_dict = {}
input_dict["global_in_2"] = in_X
input_dict["global_in"] = in_h_t_1
input_dict["global_in_1"] = in_c_t_1


output_dict_finn_tidy = oxe.execute_onnx(finn_model, input_dict,return_full_exec_context=True) #,return_full_exec_context=True
# Streamline_out = np.array(output_dict_finn_tidy.get("Mul_23_out0"))#This is h_t_inter renames
# print(Streamline_out)
print(output_dict_finn_tidy)
# Streamline_out = np.array(output_dict_finn_tidy.get("global_out"))

/home/temporary/Desktop/qlstm_finn/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor MultiThreshold_0_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/temporary/Desktop/qlstm_finn/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor MultiThreshold_1_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/temporary/Desktop/qlstm_finn/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor MultiThreshold_2_out0 can't be represented with the set datatype annotation (INT6), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/temporary/Desktop/qlstm_finn/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor MultiThreshold_3_out0 can't be represented with the set datatype annotation 

{'global_in': array([[15.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32), 'global_in_1': array([[12.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32), 'global_in_2': array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.],
       [8.],
       [9.]], dtype=float32), 'global_out': array([[  7.1922307],
       [-11.698284 ],
       [ -3.6276915],
       [ -9.907965 ],
       [-12.317261 ]], dtype=float32), 'global_out_1': array([[-0.42216498],
       [-0.6332475 ],
       [ 0.035

In [13]:
showSrc(Streamline)
from qonnx.transformation.base import Transformation
from qonnx.transformation.batchnorm_to_affine import BatchNormToAffine
from qonnx.transformation.general import (
    ConvertDivToMul,
    ConvertSubToAdd,
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
)
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.remove import RemoveIdentityOps

from finn.transformation.streamline.absorb import (
    Absorb1BitMulIntoConv,
    Absorb1BitMulIntoMatMul,
    AbsorbAddIntoMultiThreshold,
    AbsorbMulIntoMultiThreshold,
    AbsorbSignBiasIntoMultiThreshold,
    FactorOutMulSignMagnitude,
    AbsorbTransposeIntoMultiThreshold
)
from finn.transformation.streamline.collapse_repeated import (
    CollapseRepeatedAdd,
    CollapseRepeatedMul,
)
from finn.transformation.streamline.reorder import (
    MoveAddPastConv,
    MoveAddPastMul,
    MoveMulPastMaxPool,
    MoveScalarAddPastMatMul,
    MoveScalarLinearPastInvariants,
    MoveScalarMulPastConv,
    MoveScalarMulPastMatMul,
    MoveLinearPastEltwiseAdd,
    MoveLinearPastEltwiseMul,
    MoveTransposePastScalarMul,
    MoveTransposePastJoinAdd
)
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from finn.transformation.streamline.sign_to_thres import ConvertSignToThres

class Streamline(Transformation):
    """Apply the streamlining transform, see arXiv:1709.04060."""

    def apply(self, model):
        streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),
            MoveMulPastMaxPool(),
            AbsorbAddIntoMultiThreshold(),
            FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
        for tr

In [ ]:
#Setting datatypes of the input and the hidden state
# dtype = finn_model.set_tensor_datatype("global_in_2","UINT8")
# dtype = finn_model.set_tensor_datatype("global_in","INT8")

In [14]:
 ##MoveScalarMulPastMatMul(),  #Not able to apply this transformation as both the inputs are not constants; one of the inputs 
streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),     
            BatchNormToAffine(), 
            ConvertSignToThres(),  
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),  
            AbsorbSignBiasIntoMultiThreshold(), #One of the few transformations where the 'bias' is absorbed into the previous MT node and not moved to the next one.
            MoveAddPastMul(),     
            MoveScalarAddPastMatMul(), 
            MoveAddPastConv(),       
            MoveScalarMulPastConv(), 
            MoveAddPastMul(), 
            # MoveScalarMulPastMatMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),   
            MoveMulPastMaxPool(),  
            AbsorbAddIntoMultiThreshold(), 
            FactorOutMulSignMagnitude(), 
            AbsorbMulIntoMultiThreshold(), #This transformation absorbs the Scalar Mul nodes into the next Multithreshold nodes.
            MoveLinearPastEltwiseAdd(), #This transformation helps us get all the scalar mul nodes past the elstwiseadd. 
            MoveLinearPastEltwiseMul(),#This transformation helps us get scalar mul's past eltwisemuls. We can then absorb them into the multithrehsold opertion and remove them from the graph entirely.
            AbsorbMulIntoMultiThreshold(), #The scalar mul nodes passed in the previous step are now merged into the multithreshold node.
            # # Absorb1BitMulIntoMatMul(), 
            # # Absorb1BitMulIntoConv(),
            # # # MoveScalarMulPastMatMul(),  #Not able to apply this transformation as both the inputs are not constants; one of the inputs 
            # MoveLinearPastEltwiseAdd(),
            # AbsorbMulIntoMultiThreshold(),
            RoundAndClipThresholds(),
            MoveScalarMulPastMatMul(), #To move activation scales im the dense part beyond dense layers.
            AbsorbMulIntoMultiThreshold(),
            MoveLinearPastEltwiseAdd(),
            # CollapseRepeatedMul(), #Collapsing repeated sclar mul's obtained due to the above MoveScalarMulPastMatMul transformation
            AbsorbMulIntoMultiThreshold(), #For the last Multithreshold node in the graph
            RoundAndClipThresholds(),
            CollapseRepeatedMul(),
            # MoveTransposePastScalarMul(),
            # MoveTransposePastJoinAdd()
            # AbsorbTransposeIntoMultiThreshold()
            # MoveScalarLinearPastInvariants()
        ]
i = 0
for trn in streamline_transformations:
    print('Transformation = ',trn)
    finn_model = finn_model.transform(trn)
    finn_model = finn_model.transform(RemoveIdentityOps())
    finn_model = finn_model.transform(GiveUniqueNodeNames())
    finn_model = finn_model.transform(GiveReadableTensorNames())
    finn_model = finn_model.transform(InferDataTypes())
    finn_model.save('streamline_'+str(i)+'.onnx')
    i = i+1

Transformation =  <qonnx.transformation.general.ConvertSubToAdd object at 0x7f0a717bf6d0>
Transformation =  <qonnx.transformation.general.ConvertDivToMul object at 0x7f0a717bf550>
Transformation =  <qonnx.transformation.batchnorm_to_affine.BatchNormToAffine object at 0x7f0a717be410>
Transformation =  <finn.transformation.streamline.sign_to_thres.ConvertSignToThres object at 0x7f0a717be3b0>
Transformation =  <finn.transformation.streamline.reorder.MoveMulPastMaxPool object at 0x7f0a717bc820>
Transformation =  <finn.transformation.streamline.reorder.MoveScalarLinearPastInvariants object at 0x7f0a717bdc60>
Transformation =  <finn.transformation.streamline.absorb.AbsorbSignBiasIntoMultiThreshold object at 0x7f0a717bdc30>
Threshold name :  MultiThreshold_0_param0
Add Weight name :  Add_0_param0
INT8
-----------------------------------
Threshold name :  MultiThreshold_1_param0
Add Weight name :  Add_1_param0
INT8
-----------------------------------
Threshold name :  MultiThreshold_2_param0
A

/home/temporary/Desktop/qlstm_finn/finn/deps/qonnx/src/qonnx/core/modelwrapper.py:386: UserWarning: find_consumer: found multiple consumers, returning all of them
  warnings.warn("find_consumer: found multiple consumers, returning all of them")


Transformation =  <finn.transformation.streamline.reorder.MoveLinearPastEltwiseAdd object at 0x7f0a717be4d0>
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
Transformation =  <finn.transformation.streamline.reorder.MoveLinearPastEltwiseMul object at 0x7f0a717be440>
Transformation =  <finn.transformation.streamline.absorb.AbsorbMulIntoMultiThreshold object at 0x7f0a717bf5b0>
Mul_0_param0
0.035180416
Mul_1_param0
0.035180416
Mul_2_param0
0.01061632
Mul_4_param0
0.006405877
Mul_8_param0
0.00045926642
Mul_10_param0
0.029902492
Mul_11_param0
0.029902492
Mul_13_param0
0.00035575026
Mul_14_param0
0.035180416
Mul_15_param0
0.007681173
Mul_16_param0
0.008241024
Mul_17_param0
0.021023383
Mul_18_param0
0.03191494
Mul_0_param0
0.035180416
Mul_1_param0
0.035180

In [16]:
finn_model.save('./streamline_model_roundandclip.onnx')
showInNetron('./streamline_model_roundandclip.onnx')

Stopping http://0.0.0.0:8081
Serving './streamline_model_roundandclip.onnx' at http://0.0.0.0:8081


In [ ]:
showInNetron('./streamline_20.onnx')

Okay so by skipping the Mul node to apply all the transformations that wwere throwing errors that graph has now become smaller. This is mainly attributed to the AbsorbMulIntoMultithreshold transformation. But still certain Mul operators are not getting absorbed into the Multithreshold node. 

The modified MoveLinearPastEltwiseAdd() transformation is not functionally coherent. Becuase I  think when the muls are moved around the multiplication is not accounted for. The transformation needs to be modified to incorporate the multiplication operator in this case.

In [ ]:
in_X = np.zeros([10,1],dtype=np.float32).reshape([10,1])
# in_X[0][0] = 0
# in_X[1][0] = 1
# in_X[2][0] = 2
# in_X[3][0] = 3
# in_X[4][0] = 4
# in_X[5][0] = 5
# in_X[6][0] = 6
# in_X[7][0] = 7
# in_X[8][0] = 8
# in_X[9][0] = 9
# in_h_t_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
# in_c_t_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
# in_h_t_1 = np.ones([20,1],dtype=np.float32).reshape([20,1])
# in_c_t_1 = np.ones([20,1],dtype=np.float32).reshape([20,1])
# in_h_t_1[0][0] = 15
# in_c_t_1[0][0] = 12

in_h_t_1 = np.zeros([20,1],dtype=np.float32).reshape([20,1])
in_c_t_1 = np.zeros([20,1],dtype=np.float32).reshape([20,1])


in_h_t_1 = Streamline_out
in_c_t_1 = Streamline_out_ct


input_dict = {}
input_dict["global_in_2"] = in_X
input_dict["global_in"] = in_h_t_1
input_dict["global_in_1"] = in_c_t_1

output_dict_finn_streamlined = oxe.execute_onnx(finn_model, input_dict,return_full_exec_context=True) #,return_full_exec_context=True
Streamline_out = np.array(output_dict_finn_streamlined.get("global_out_1"))
print(Streamline_out)
Streamline_out_ct = np.array(output_dict_finn_streamlined.get("global_out_2"))
print(Streamline_out_ct)
print(output_dict_finn_streamlined)

In [ ]:
output_dict_finn_streamlined.get("MultiThreshold_18_out0")

In [ ]:
output_dict_finn_streamlined.get("MatMul_8_out0")

In [ ]:
output_dict_finn_streamlined.get("MultiThreshold_19_out0")

In [ ]:
output_dict_finn_streamlined.get("Add_5_out0")

In [ ]:
output_dict_finn_streamlined.get("Mul_3_out0")

In [ ]:
print(QONNX_out - Streamline_out)
print(QONNX_out_ct - Streamline_out_ct)

In [ ]:
in_h_t_1 = Streamline_out
in_c_t_1 = Streamline_out_ct

In [ ]:
showInNetron('./prev_streamline_model.onnx')

In [ ]:
showInNetron('./lstm_relu_model_brevitas_export.onnx')

In [ ]:
# x1 = np.array(output_dict_finn.get("h_t_inter")) #i_t_dql1
# print(x1)
# x2 = np.array(output_dict_finn_streamlined.get("global_out_1")) #i_t_dql1
# print(x2*128)
# print("----------------------")
# print(x1-x2)

In [ ]:
# test_out = np.array(output_dict_finn_streamlined.get("Add_4_out0"))
# print(test_out)
# test_out = np.array(output_dict_finn_streamlined.get("MultiThreshold_16_out0"))
# print(test_out)
# test_out = np.array(output_dict_finn_streamlined.get("Mul_2_out0"))
# print(test_out)
test_out = np.array(output_dict_finn_streamlined.get("MatMul_9_out0"))
print(test_out)

So, More streamlinind needs to be done on the complete IDS graph.
1. Set the input datatype to UINT8 instead of FLOAT32 : This part done. Some Mul's are not getting absorbed into the MT node. Need to pass them.
2. Streamline the Dense part of the graph. Preferrrably remove the Transpose operation from the graph. Then that would allow more nodes especially mul nodes to be merged into one another.
3. That would reduce the overall computation of the graph and help in reducing the resource utilisation of the graph.

In [ ]:
test_out = np.array(output_dict_finn_streamlined.get("Add_8_out0"))
print(test_out)
# test_out = np.array(output_dict_finn_streamlined.get("MultiThreshold_2_out0"))
# print(test_out)
# test_out = np.array(output_dict_finn_streamlined.get("MultiThreshold_11_out0"))
# print(test_out)

In [ ]:
x3 = np.array(output_dict_finn_streamlined.get("global_out"))   #h_t
x4 = np.array(output_dict_finn_streamlined.get("global_out_1")) #c_t
print(x3)
print('-----------------------------')
print(x)
# print(x4)
scale = 0.0078125
difference = x - x3
print("Absolute difference")
print(difference)
print("-----------------------------")
difference = difference/scale
print("------------------------------")
print(difference)

In [ ]:
# Trying to execute the above streamlined ONNX graph and comparing against the QONNX and before streamlining FINN-ONNX representation.
# in_X = np.array([1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
# in_h_t_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
# in_c_t_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
in_h_t_1 = np.zeros([20,1],dtype=np.float32)
in_c_t_1 = np.zeros([20,1],dtype=np.float32)
in_h_t_1[0][0] = 0; in_h_t_1[1][0] = -110; in_h_t_1[2][0] = 0; in_h_t_1[3][0] = 0; in_h_t_1[4][0] = 0; in_h_t_1[5][0] = 0; in_h_t_1[6][0] = -110; in_h_t_1[7][0] = -110; in_h_t_1[8][0] = 0; in_h_t_1[9][0] = 0;
in_h_t_1[10][0] = 107; in_h_t_1[11][0] = 0; in_h_t_1[12][0] = -56; in_h_t_1[13][0] = 0; in_h_t_1[14][0] = 107; in_h_t_1[15][0] = 107; in_h_t_1[16][0] = 0; in_h_t_1[17][0] = 0; in_h_t_1[18][0] = 0; in_h_t_1[19][0] = 0;
in_c_t_1[0][0] = 0; in_c_t_1[1][0] = -1.2486; in_c_t_1[2][0] = 1.2486; in_c_t_1[3][0] = -1.2486; in_c_t_1[4][0] = 0; in_c_t_1[5][0] = 0; in_c_t_1[6][0] = -1.2486; in_c_t_1[7][0] = -1.2486; in_c_t_1[8][0] = 0; in_c_t_1[9][0] = 0;
in_c_t_1[10][0] = 1.2486; in_c_t_1[11][0] = 0; in_c_t_1[12][0] = -1.2486; in_c_t_1[13][0] = -1.2486; in_c_t_1[14][0] = 1.2486; in_c_t_1[15][0] = 1.2486; in_c_t_1[16][0] = 0; in_c_t_1[17][0] = 0; in_c_t_1[18][0] = 1.2486; in_c_t_1[19][0] = 1.2486;


# in_X = np.array([1,2,3,4,5,-6,-7,-8,-9,-10],dtype=np.float32)
# in_X = np.ones([10,1],dtype=np.float32)
# in_X = np.zeros([10,1],dtype=np.float32)
in_X = in_X.reshape([10,1])
in_h_t_1 = in_h_t_1.reshape([20,1])
in_c_t_1 = in_c_t_1.reshape([20,1])

input_dict = {}
input_dict["global_in_2"] = in_X
input_dict["global_in"] = in_h_t_1
input_dict["global_in_1"] = in_c_t_1

output_dict_finn_streamlined = oxe.execute_onnx(finn_model, input_dict,return_full_exec_context=True) #,return_full_exec_context=True
print(output_dict_finn_streamlined)
Streamline_out = np.array(output_dict_finn_streamlined.get("global_out"))